In [85]:
import numpy as np
from scipy.optimize import linprog

# Background data
blend_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
smalls_names = ['Smalls-A', 'Smalls-B', 'Smalls-C', 'Smalls-D', 'Smalls-E', 'Smalls-F', 'Smalls-G']
unit_prices = np.array([7.3, 9.2, 7.4, 7.7, 11.1, 13.7, 8.6])

# Allowable (max) Smalls addback quantities per batch of Blend operation
max_addback = {
    'Smalls-A': [20, 20, 20, 20, 20, 20, 20],
    'Smalls-B': [0, 45, 0, 0, 0, 0, 0],
    'Smalls-C': [0, 0, 40, 25, 30, 0, 0],
    'Smalls-D': [50, 45, 0, 60, 0, 25, 0],
    'Smalls-E': [0, 0, 0, 0, 60, 20, 30],
    'Smalls-F': [25, 0, 0, 0, 0, 60, 40],
    'Smalls-G': [0, 30, 0, 0, 0, 0, 50]
}

# Profit/Loss calculation
profit_loss = np.zeros((7, 7))
for i in range(7):
    for j in range(7):
        profit_loss[i, j] = unit_prices[j] - unit_prices[i]

# Input Smalls
available_smalls = np.array([float(input(f"Enter available quantity of {smalls}: ")) for smalls in smalls_names])

# Input Blends
planned_batches = np.array([int(input(f"Enter planned number of batch operations for Blend {blend}: ")) for blend in blend_names])

# Objective function coefficients
c = -profit_loss.flatten()

# Constraints for smalls (inequality constraints)
A_ub = []
b_ub = []

for i, smalls in enumerate(smalls_names):
    row = np.zeros(49)
    for j in range(7):
        row[i * 7 + j] = 1  # Each smalls can contribute fully to one blend's batch
    A_ub.append(row)
    b_ub.append(available_smalls[i])  # Available quantity for each small

A_ub = np.array(A_ub)
b_ub = np.array(b_ub)

# Bounds (0 to max_addback * planned_batches)
bounds = [(0, max_addback[smalls][j] * planned_batches[j]) for smalls in smalls_names for j in range(7)]

# Solving the linear programming problem using inequality constraints
result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')

# Initial check
if result.success:
    # Returning the solution
    solution = np.round(result.x.reshape((7, 7)), decimals=2)

    # Calculating total monetary gain/loss
    total_gain_loss = np.sum(solution * profit_loss)

    # Calculating remaining quantities of Smalls
    remaining_smalls = available_smalls - np.sum(solution, axis=1)

    # The addback plan
    print("\nOptimal Addback Plan:")
    for i, smalls in enumerate(smalls_names):
        for j, blend in enumerate(blend_names):
            if solution[i, j] > 0:
                print(f"Add {solution[i, j]:.2f} KG of {smalls} to Blend {blend}")

    # Total monetary gain/loss
    print(f"\nTotal Monetary Gain/Loss: {total_gain_loss:.2f} GBP")

    # Remaining quantities of Smalls
    print("\nRemaining Quantities of Smalls:")
    for i, smalls in enumerate(smalls_names):
        print(f"{smalls}: {remaining_smalls[i]:.2f} KG")
else:
    print("Optimization failed. Please check the input data and constraints.")

Enter available quantity of Smalls-A:  40
Enter available quantity of Smalls-B:  0
Enter available quantity of Smalls-C:  40
Enter available quantity of Smalls-D:  0
Enter available quantity of Smalls-E:  80
Enter available quantity of Smalls-F:  0
Enter available quantity of Smalls-G:  0
Enter planned number of batch operations for Blend A:  0
Enter planned number of batch operations for Blend B:  0
Enter planned number of batch operations for Blend C:  0
Enter planned number of batch operations for Blend D:  1
Enter planned number of batch operations for Blend E:  2
Enter planned number of batch operations for Blend F:  3
Enter planned number of batch operations for Blend G:  1



Optimal Addback Plan:
Add 40.00 KG of Smalls-A to Blend F
Add 40.00 KG of Smalls-C to Blend E
Add 60.00 KG of Smalls-E to Blend F

Total Monetary Gain/Loss: 560.00 GBP

Remaining Quantities of Smalls:
Smalls-A: 0.00 KG
Smalls-B: 0.00 KG
Smalls-C: 0.00 KG
Smalls-D: 0.00 KG
Smalls-E: 20.00 KG
Smalls-F: 0.00 KG
Smalls-G: 0.00 KG
